# Logistic Regression Implementation

An example binary classification problem can be represented by a dataset containing information about customers who did or did not default on their credit cards.  We want to do the following:

- Basic EDA: explore default groups for each individual feature (boxplots could be a nice way in here)
- Process categorical variables using `pd.get_dummies`
- Split your data
- Run a `LogisticRegression` to explore the likelihood of default based on the `balance` column.
- Cross validate this using values $[0.1, 1, 5, 10, 100]$ for the `C` parameter.
- Incorporate `PolynomialFeatures` into your model and rerun.  How did the performance change?
- Repeat for the `student` column.

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

In [12]:
df = pd.read_csv('data/default.csv')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
default    10000 non-null object
student    10000 non-null object
balance    10000 non-null float64
income     10000 non-null float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [38]:
df = pd.get_dummies(df, drop_first = True)
#DONT FORGET TO DROP FIRST DUMMIES !!

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
balance        10000 non-null float64
income         10000 non-null float64
default_No     10000 non-null uint8
default_Yes    10000 non-null uint8
student_No     10000 non-null uint8
student_Yes    10000 non-null uint8
dtypes: float64(2), uint8(4)
memory usage: 195.4 KB


In [ ]:
#don't forget to do EDA before jumping into the below !
#look at answers to see

In [ ]:
#also - do a pred without the gridsearch first to familiarize

In [23]:
X = df.balance.values.reshape(-1,1)
y = df.default_Yes

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [29]:
pipe = make_pipeline(PolynomialFeatures(), LogisticRegression())
params = {'polynomialfeatures__degree': [i for i in range(1,5)], 'logisticregression__C': [.1, 1, 5, 10, 100]}
grid = GridSearchCV(pipe, param_grid=params, cv = 5)

In [31]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'polynomialfeatures__degree': [1, 2, 3, 4], 'logisticregression__C': [0.1, 1, 5, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [34]:
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
best = grid.best_estimator_
pred = best.predict(X_test)

In [37]:
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.99      0.98      0.99      2447
          1       0.41      0.60      0.48        53

avg / total       0.98      0.97      0.98      2500

